# Rivercats - Extended Feature Selection
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Feb 22, 2022

## Hypothesis
Two very important components of a machine learning model are feature selection and feature engineering. Our idea is that adding some more features to the StellarAlgo retention model could improve performance of the model. This notebook will test the standard StellarAlgo retention model features.

## Experiment
This section details our experiment including querying data, data transformations, feature selection and modelling.

In [1]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from pycaret.classification import *

warnings.filterwarnings('ignore')

### Let's connect to MSSQL and run a stored proc to get our dataset:

In [2]:
# connect to SQL Server.
SERVER = '34.206.73.189' 
DATABASE = 'datascience' 
USERNAME = 'nrad' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

In [3]:
lkupclientid = 12 # Rivercats

cursor = CNXN.cursor()

storedProc = (
    f"""Exec [stlrMILB].[ds].[getRetentionScoringModelData] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

# apply some data transformations
df["year"] = pd.to_numeric(df["year"])

CNXN.commit()
cursor.close()

df.head()
#df.info()

,lkupClientId,dimCustomerMasterId,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,click_link,fill_out_form,open_email,send_email,unsubscribe_email,openToSendRatio,clickToSendRatio,clickToOpenRatio,posting_records,resale_records,resale_atp,forward_records,cancel_records,email,inbound_email,inbound_phonecall,inperson_contact,internal_note,left_message,outbound_email,outbound_phonecall,phonecall,text,unknown,gender,childrenPresentInHH,maritalStatus,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,credits_after_refund,is_Lockdown,NumberofGamesPerSeason,CNTPostponedGames,isNextYear_Buyer
0,12,322128315,4220830,2016,Full Season,5616.0,2016-08-31,0.417857,38,TRUE,2190,181,22.72,35,4,5,2,5,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,99999,College Certificate/Diploma,None,0.0,0,74,None,0
1,12,322128618,3872116,2016,Quarter Season,756.0,2016-09-04,0.411765,81,TRUE,2555,222,3.67,7,0,0,2,2,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,99999,College Certificate/Diploma,None,0.0,0,74,None,0
2,12,322128710,3117031,2016,Quarter Season,702.0,2016-08-12,0.500000,38,TRUE,3650,174,18.59,7,3,3,1,1,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,99999,College Certificate/Diploma,None,0.0,0,74,None,1
3,12,322132005,1631005,2016,Full Season,5184.0,2016-09-05,0.521429,172,TRUE,5110,315,17.57,43,0,6,1,7,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,99999,College Certificate/Diploma,None,0.0,0,74,None,0
4,12,322133534,4296029,2016,Half Season,504.0,1970-01-01,0.000000,3,TRUE,2190,122,27.56,0,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,99999,College Certificate/Diploma,None,0.0,0,74,None,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lkupClientId            6435 non-null   int64  
 1   dimCustomerMasterId     6435 non-null   int64  
 2   customerNumber          6435 non-null   object 
 3   year                    6435 non-null   int64  
 4   productGrouping         6435 non-null   object 
 5   totalSpent              6435 non-null   float64
 6   recentDate              6435 non-null   object 
 7   attendancePercent       6435 non-null   float64
 8   renewedBeforeDays       6435 non-null   int64  
 9   isBuyer                 6435 non-null   object 
 10  source_tenure           6435 non-null   int64  
 11  tenure                  6435 non-null   int64  
 12  distToVenue             6435 non-null   float64
 13  totalGames              6435 non-null   int64  
 14  recency                 6435 non-null   

### Let's also drop the features that only have a single value, as they won't add much differentiation to our model:

In [5]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
        
df.shape

(6435, 25)

### We should also drop features that have a low correlation with the target label as they won't be useful for prediction, we'll only keep features that have a correlation above a set threshold:

In [6]:
cor = df.corr()

threshold = 0.05

#Correlation with output variable
cor_target = abs(cor["isNextYear_Buyer"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > threshold]

feats = []
for name, val in relevant_features.items():
    feats.append(name)

df_correlated = df[feats]

df_correlated.shape

df_correlated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   dimCustomerMasterId     6435 non-null   int64  
 1   year                    6435 non-null   int64  
 2   totalSpent              6435 non-null   float64
 3   attendancePercent       6435 non-null   float64
 4   renewedBeforeDays       6435 non-null   int64  
 5   source_tenure           6435 non-null   int64  
 6   totalGames              6435 non-null   int64  
 7   recency                 6435 non-null   int64  
 8   missed_games_1          6435 non-null   int64  
 9   missed_games_2          6435 non-null   int64  
 10  missed_games_over_2     6435 non-null   int64  
 11  NumberofGamesPerSeason  6435 non-null   int64  
 12  isNextYear_Buyer        6435 non-null   int64  
dtypes: float64(2), int64(11)
memory usage: 653.7 KB


### Now that we have the right features we can look at the correlations between them, if features are highly correlated with each other it might negatively impact the model:

In [7]:
corr = df_correlated.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,dimCustomerMasterId,year,totalSpent,attendancePercent,renewedBeforeDays,source_tenure,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,NumberofGamesPerSeason,isNextYear_Buyer
dimCustomerMasterId,1.00,0.28,0.19,0.03,0.09,0.21,0.21,-0.04,0.14,0.15,0.18,0.22,0.32
year,0.28,1.00,-0.09,-0.01,-0.40,-0.15,-0.12,0.00,-0.11,-0.09,-0.08,0.80,-0.16
totalSpent,0.19,-0.09,1.00,-0.15,0.30,0.32,0.72,0.03,0.48,0.52,0.68,-0.07,0.19
attendancePercent,0.03,-0.01,-0.15,1.00,-0.07,-0.13,0.24,-0.41,0.14,-0.07,-0.45,0.01,0.09
renewedBeforeDays,0.09,-0.40,0.30,-0.07,1.00,0.36,0.37,0.01,0.29,0.26,0.32,-0.27,0.15
source_tenure,0.21,-0.15,0.32,-0.13,0.36,1.00,0.29,0.05,0.22,0.25,0.37,-0.13,0.21
totalGames,0.21,-0.12,0.72,0.24,0.37,0.29,1.00,-0.17,0.75,0.60,0.46,-0.09,0.25
recency,-0.04,0.00,0.03,-0.41,0.01,0.05,-0.17,1.00,-0.14,-0.05,0.16,-0.03,-0.05
missed_games_1,0.14,-0.11,0.48,0.14,0.29,0.22,0.75,-0.14,1.00,0.53,0.28,-0.08,0.17
missed_games_2,0.15,-0.09,0.52,-0.07,0.26,0.25,0.60,-0.05,0.53,1.00,0.42,-0.07,0.15


### In order to compare two sets of features, we need to create some datasets for training and evalution:


In [8]:
# select % of the data for training
df_train = df_correlated.sample(frac=0.8, random_state=786).reset_index(drop=True)

# create the eval datasets for A and B
df_eval = df_correlated.drop(df_train.index).reset_index(drop=True)

# print out the number of records for training and eval
print('Data for Modeling: ' + str(df_train.shape))
print('Unseen Data For Predictions: ' + str(df_eval.shape), end="\n\n")

Data for Modeling: (5148, 13)
Unseen Data For Predictions: (1287, 13)



### Now we can model the data using a binary classification prediction for the isnextyear_buyer field to see how likely a customer is to re-purchase.

In [9]:
setup(
    data= df_train, 
    target="isNextYear_Buyer", 
    train_size = 0.80,
    data_split_shuffle=True,
    silent=True,
    numeric_features=[
        "totalSpent",
        "attendancePercent",
        "renewedBeforeDays",
        "source_tenure",
        "totalGames",
        "recency",
        "missed_games_1",
        "missed_games_2",
        "missed_games_over_2",
        "NumberofGamesPerSeason"
    ]
)

,Description,Value
0,session_id,6015
1,Target,isNextYear_Buyer
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(5148, 13)"
5,Missing Values,False
6,Numeric Features,11
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


('clf-default-name',
 {'USI',
  'X',
  'X_test',
  'X_train',
  '_all_metrics',
  '_all_models',
  '_all_models_internal',
  '_available_plots',
  '_gpu_n_jobs_param',
  '_internal_pipeline',
  '_ml_usecase',
  'create_model_container',
  'data_before_preprocess',
  'display_container',
  'exp_name_log',
  'experiment__',
  'fix_imbalance_method_param',
  'fix_imbalance_param',
  'fold_generator',
  'fold_groups_param',
  'fold_groups_param_full',
  'fold_param',
  'fold_shuffle_param',
  'gpu_param',
  'html_param',
  'imputation_classifier',
  'imputation_regressor',
  'iterative_imputation_iters_param',
  'log_plots_param',
  'logging_param',
  'master_model_container',
  'n_jobs_param',
  'prep_pipe',
  'pycaret_globals',
  'seed',
  'stratify_param',
  'target_param',
  'transform_target_method_param',
  'transform_target_param',
  'y',
  'y_test',
  'y_train'},
 False,
 True,
 False,
 {'acc': <pycaret.containers.metrics.classification.AccuracyMetricContainer at 0x7fe71681be80>,
 

In [10]:
model_matrix = compare_models(
    fold=10,
    include=["lr", "xgboost"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7975,0.8946,0.7940,0.7830,0.7881,0.5942,0.5948,0.4490
lr,Logistic Regression,0.5262,0.3497,0.0000,0.0000,0.0000,0.0000,0.0000,0.3060


In [11]:
best_model = create_model(model_matrix)
final_model = finalize_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8155,0.9075,0.7846,0.8182,0.8010,0.6292,0.6297
1,0.8252,0.9121,0.8051,0.8220,0.8135,0.6491,0.6492
2,0.8204,0.9100,0.8000,0.8168,0.8083,0.6394,0.6395
3,0.7888,0.8921,0.7590,0.7872,0.7728,0.5757,0.5760
4,0.8107,0.9027,0.8051,0.7970,0.8010,0.6205,0.6205
5,0.7646,0.8687,0.7795,0.7379,0.7581,0.5291,0.5299
6,0.7937,0.8940,0.7846,0.7806,0.7826,0.5863,0.5863
7,0.7767,0.8671,0.7755,0.7600,0.7677,0.5528,0.5529
8,0.7981,0.9024,0.8256,0.7667,0.7951,0.5966,0.5982
9,0.7810,0.8892,0.8205,0.7442,0.7805,0.5631,0.5657


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

## Observations
Here you can document some ideas on the results from above

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?